In [1]:
import keras
keras.__version__

'2.6.0'

# 5-1 합성곱 신경망

## 간단한 컨브넷 만들기 

In [2]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


- Conv2D, MaxPooling2D 출력: (height,width,channels) 크기의 3D 텐서
- 높이, 넓이 차원은 네트워크가 깊어질수록 작아지는 경향 

## 컨브넷 위에 분류기 추가 

In [4]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [7]:
#MNIST를 이용한 컨브넷 훈련


from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [8]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 22s 22ms/step - loss: 0.1777 - accuracy: 0.9446
Epoch 2/5
938/938 [==============================] - 22s 23ms/step - loss: 0.0501 - accuracy: 0.9844
Epoch 3/5
938/938 [==============================] - 22s 23ms/step - loss: 0.0331 - accuracy: 0.9901
Epoch 4/5
938/938 [==============================] - 22s 23ms/step - loss: 0.0255 - accuracy: 0.9921
Epoch 5/5
938/938 [==============================] - 22s 23ms/step - loss: 0.0207 - accuracy: 0.9937


In [9]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 2s 6ms/step - loss: 0.0264 - accuracy: 0.9917


In [10]:
test_acc

0.9916999936103821

## 합성곱 연산

- Dense층은 입력 특성 구간에 있는 전역 패턴을 학습하지만 합성곱 층은 지역 패턴을 학습한다. 
- 컨브넷 성질
    - 학습된 패턴은 평행이동 불변설을 가진다. 
    - 컨브넷은 패턴의 공간적 계층 구조를 학습할 수 있다. 
- 합성곱 연산은 특성맵이라곱 부르는 3D텐서에 적용
    - 텐서는 2개의 공간축(높이와 너비)과 깊이 축(채널 축)으로 구성
    - RGB 이미지는 3개의 컬러(빨강, 초록, 파랑)을 가지므로 깇이 축의 차원은 3이 된다. 
- 합성곱 연산은 입력 특성맵에서 작은 패치를 추출 --> 이런 모든 패치에 같은 변환을 적용해 출력 특성맵을 만든다. 
- 합성곱의 핵심적인 2개의 파라미터
    - 입력으로부터 뽑아낼 패치의 크기: 전형적으로 3*3 또는 5*5 사용
    - 특성 맵의 출력 깊이: 합성곱으로 계산할 필터의 수
    
    
## 합성곱 스트라이드 이해

- 출력 크기에 영향을 미치는 요소
- 두 번 연속적인 윈도우 사이의 거리가 스트라이드라고 불리는 합성곱의 파라미터
- 기본값은 1, 큰 스트라이드 합성곱도 가능


## 최대 풀링 연산
- MaxPooling2D 층마다 절반으로 줄어든다. 이것이 합성곱과 매우 비슷하게 강제적으로 특성 맵을 다운 샘플링하는 것이 최대 풀링의 역할
- 최대 풀링은 입력 특성 맵에서 윈도우에 맞는 패치를 추출하고 각 채널별로 최대값을 출력
- 합성곱과 개념적으로 비슷하지만 추출한 패치에 학습된 선형 변환(합성곱 커널)을 적용하는 대신 하드코딩된 최댓값 추출 연산을 사용
